Prep Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.sets import load_sets

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

In [4]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

Baseline Model

In [5]:
from src.models.null import NullModel
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [6]:
from src.models.performance import print_class_perf
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.07444192974099043
F1 Training: 0.010315310209270104


Define Architecture

This versin uses the model modified from lab5

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_out = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)
    
model = PytorchMultiClass(X_train.shape[1])

In [33]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

Train Model

In [28]:
import torch
import torch.nn as nn

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [35]:
N_EPOCHS = 20
BATCH_SIZE = 1000

In [36]:
#use train_classification, test_classification defined in lab5
from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 1
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 2
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 3
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 4
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 5
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 6
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 7
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 8
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 9
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 10
	(train)	|	Loss: 0.0046	|	Acc: 1.0%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 11
	(train)	|	Loss: 0.0046	|	Acc: 1.1%
	(valid)	|	Loss: 0.0046	|	Acc: 0.8%
Epoch: 12
	(train)	|	Loss:

In [38]:
# torch.save(model, "../models/pytorch_nn_v1.pt")

In [34]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0005	|	Accuracy: 0.0


try other models